<a href="https://colab.research.google.com/github/Gideon-art/Hamoye/blob/main/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

energy_data = pd.read_csv('/content/energydata_complete.csv')

energy_data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
from sklearn.linear_model import LinearRegression
import numpy as np

# select the relevant columns for the linear regression
X = energy_data[["T2"]]
y = energy_data["T6"]

# create a linear regression model
model = LinearRegression()

# fit the model to the data
model.fit(X, y)

# calculate R^2 value and round to 2 dp
r_squared = round(model.score(X, y), 2)

# print the R^2 value
r_squared

0.64

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# remove the specified columns from the dataframe
energy_data = energy_data.drop(["date", "lights"], axis=1)

# create the MinMaxScaler object
scaler = MinMaxScaler()

# normalize the dataset
energy_data_norm = pd.DataFrame(scaler.fit_transform(energy_data), columns=energy_data.columns)

# set the target variable
y = energy_data_norm["Appliances"]

# set the predictor variables
X = energy_data_norm.drop("Appliances", axis=1)

# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# create a linear regression model
model = LinearRegression()

# fit the model to the training data
model.fit(X_train, y_train)

# evaluate the model on the test data
score = model.score(X_test, y_test)

# print the R^2 value
score

0.14890246319303535

In [6]:
from sklearn.metrics import mean_absolute_error

#test set predictions
y_pred = model.predict(X_test)

# mean absolute error
mae = round(mean_absolute_error(y_test, y_pred), 2)

# display the mean absolute error
mae

0.05

In [7]:
# calculate the residual sum of squares (RSS) on the test set
rss = round(((y_test - y_pred) ** 2).sum(), 2)

# display the RSS
rss

45.35

In [8]:
from sklearn.metrics import mean_squared_error

# calculate the root mean squared error (RMSE) on the test set
rmse = mean_squared_error(y_test, y_pred, squared=False)

# display the RMSE rounded to 3 decimal places
round(rmse, 3)

0.088

In [9]:
from sklearn.metrics import r2_score

# calculate the coefficient of determination (R^2) on the test set
r2 = r2_score(y_test, y_pred)

# R^2
round(r2, 2)

0.15

In [13]:
# obtain the feature weights from the linear model
weights = pd.Series(model.coef_, index=X.columns)

# display the feature weights sorted in ascending order
weights_sorted = weights.sort_values()
print("Features with the lowest weights:")
print(weights_sorted.head())

# display the feature weights sorted in descending order
print("\nFeatures with the highest weights:")
print(weights_sorted.tail())

Features with the lowest weights:
RH_out       -0.049557
RH_8         -0.000110
T1            0.000000
Tdewpoint     0.000000
Visibility    0.000000
dtype: float64

Features with the highest weights:
T2           0.000000
RH_6        -0.000000
rv2         -0.000000
Windspeed    0.002912
RH_1         0.017880
dtype: float64


In [10]:
from sklearn.linear_model import Ridge


# train a ridge regression model with alpha=0.4
model = Ridge(alpha=0.4)
model.fit(X_train, y_train)

# make predictions on the test set
y_pred = model.predict(X_test)

# calculate the root mean squared error (RMSE) on the test set
rmse = mean_squared_error(y_test, y_pred, squared=False)

# display the RMSE rounded to 3 decimal places
round(rmse, 3)

0.088

In [11]:
from sklearn.linear_model import Lasso

# train a lasso regression model with alpha=0.001
model = Lasso(alpha=0.001)
model.fit(X_train, y_train)

# get the feature weights
feature_weights = model.coef_

# count the number of non-zero feature weights
num_nonzero_weights = sum(feature_weights != 0)

# display the number of non-zero feature weights
num_nonzero_weights

4

In [12]:

# train a lasso regression model with alpha=0.001
model = Lasso(alpha=0.001)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# calculate the root mean squared error (RMSE) on the test set
rmse = mean_squared_error(y_test, y_pred, squared=False)

# display the RMSE rounded to 3 decimal places
round(rmse, 3)

0.094